In [1]:
import pandas as pd
import os

In [2]:
import numpy as np

In [3]:
ROOT = os.path.abspath('/home/mod/Workspace/github/facebook-hackathon/data/')

In [4]:
fake_data = pd.read_csv(os.path.join(ROOT, 'fake_cleaned.csv'))

In [5]:
fake_data.head()

,text,label
0,Print They should pay all the back all the mon...,fake
1,Why Did Attorney General Loretta Lynch Plead T...,fake
2,Red State : \nFox News Sunday reported this mo...,fake
3,Email Kayla Mueller was a prisoner and torture...,fake
4,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,fake


In [6]:
fake_data = fake_data.dropna()

In [7]:
indices = []
for idx, row in fake_data.iterrows():
    if len(row.text) < 30:
        indices.append(idx)
#         print(idx, row.text)

In [8]:
fake_data = fake_data.drop(fake_data.index[indices])

In [9]:
fake_data['label'] = fake_data['label'].apply(lambda x: 1 if x=='fake' else 0)

In [10]:
fake_data['text'] = fake_data['text'].replace('\n','', regex=True)

In [11]:
fake_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28463 entries, 0 to 28664
Data columns (total 2 columns):
text     28463 non-null object
label    28463 non-null int64
dtypes: int64(1), object(1)
memory usage: 667.1+ KB


In [12]:
# fake_data.to_csv(os.path.join(ROOT, 'fake_dropped.csv'), index=False)

In [13]:
msk = np.random.rand(len(fake_data)) < 0.7

train = fake_data[msk]
test = fake_data[~msk]

In [14]:
test.label.values

array([1, 1, 1, ..., 0, 0, 0])

In [15]:
msk = np.random.rand(len(train)) < 0.7
val = train[~msk]

In [16]:
val.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5936 entries, 0 to 28664
Data columns (total 2 columns):
text     5936 non-null object
label    5936 non-null int64
dtypes: int64(1), object(1)
memory usage: 139.1+ KB


In [17]:
# val.to_csv(os.path.join(ROOT, 'fake_cleaned_val.csv'), index=False)

In [18]:
# train.to_csv(os.path.join(ROOT, 'fake_cleaned_train.csv'), index=False)

In [19]:
# test.to_csv(os.path.join(ROOT, 'fake_cleaned_test.csv'), index=False)

In [15]:
from torchtext import data
import spacy

In [16]:
NLP = spacy.load('en_core_web_sm')
tokenizer = lambda sent: [x.text for x in NLP.tokenizer(sent) if x.text != " "]

In [17]:
# Convert DataFrame to torchtext.data.Dataset
text = data.Field(
    sequential=True, tokenize=tokenizer, lower=True,
    fix_length=200,
)
label = data.Field(sequential=False, use_vocab=False)
data_fields = [("text", text), ("label", label)]

In [18]:
train_examples = [data.Example.fromlist(i, data_fields) 
                  for i in train.values.tolist()]

In [19]:
train_data = data.Dataset(train_examples, data_fields)

In [20]:
text.build_vocab(train_data)

In [21]:
vocab = text.vocab

In [40]:
train_data

In [32]:
train_iterator = data.BucketIterator(
            train_data,
            batch_size=16,
            sort_key=lambda x: len(x.text),
            repeat=False,
            shuffle=True)

In [33]:
val_examples = [data.Example.fromlist(i, data_fields) 
                  for i in val.values.tolist()]

In [34]:
val_data = data.Dataset(val_examples, data_fields)

In [35]:
val_iterator = data.BucketIterator(
            val_data,
            batch_size=16,
            sort_key=lambda x: len(x.text),
            repeat=False,
            shuffle=False)

In [66]:
data = []
labels = []
with io.open('/home/mod/Workspace/github/facebook-hackathon/data/fake_cleaned_test.csv', encoding="utf8") as f:
    reader = unicode_csv_reader(f)
    for row in reader:
#         corpus = ' '.join(row[1:])
#         token_ids = list(sp_generator([corpus]))[0]
        print(row)
        tokens = ' '.join(row[0:])
        print(tokens)
        tokens = tokenizer(tokens)
        print(row[1])
        label = int(row[1])
        data.append((label, torch.tensor(tokens)))
        labels.append(label)

['text', 'label']
label
label


ValueError: invalid literal for int() with base 10: 'label'

In [155]:
# df = pd.read_csv('/home/mod/Workspace/github/facebook-hackathon/data/fake_cleaned_test.csv')
df = train

In [158]:
tokenizer = get_tokenizer("basic_english")

In [161]:
data = []
labels = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    tokens = tokenizer(row.text)
    tokens = torch.tensor([vocab[token] for token in tokens])
    label = int(row.label)
    data.append((label, torch.tensor(tokens)))
    labels.append(label)

  0%|          | 0/8589 [00:00<?, ?it/s]/home/mod/.local/share/virtualenvs/Workspace-lCUZRoor/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
100%|██████████| 8589/8589 [00:01<00:00, 8391.10it/s]


In [42]:
import torch
import torchtext
from torchtext.datasets import text_classification

NGRAMS = 2

# if not os.path.isdir('./data'):
#     os.mkdir('./data')

# train_dataset, test_dataset = text_classification.DATASETS['AG_NEWS'](
#     root='./data', ngrams=NGRMS, vocab=None)
# BATCH_SIZE = 16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [157]:
import logging
import torch
import io
from torchtext.utils import download_from_url, extract_archive, unicode_csv_reader
from torchtext.data.utils import ngrams_iterator
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.vocab import Vocab
from tqdm import tqdm

In [4]:
??build_vocab_from_iterator

In [51]:
BATCH_SIZE=16

In [52]:
len(vocab)

184587

In [44]:
import torch.nn as nn
import torch.nn.functional as F
class TextSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [45]:
# VOCAB_SIZE = len(train_dataset.get_vocab())
VOCAB_SIZE = len(vocab)
EMBED_DIM = 32
# NUN_CLASS = len(train_dataset.get_labels())
NUN_CLASS = 2
model = TextSentiment(VOCAB_SIZE, EMBED_DIM, NUN_CLASS).to(device)

In [46]:
def generate_batch(batch):
    label = torch.tensor([entry[0] for entry in batch])
    text = [entry[1] for entry in batch]
    offsets = [0] + [len(entry) for entry in text]
    # torch.Tensor.cumsum returns the cumulative sum
    # of elements in the dimension dim.
    # torch.Tensor([1.0, 2.0, 3.0]).cumsum(dim=0)

    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text = torch.cat(text)
    return text, offsets, label

In [55]:
from torch.utils.data import DataLoader

def train_func(sub_train_):

    # Train the model
    train_loss = 0
    train_acc = 0
    data = DataLoader(sub_train_, batch_size=BATCH_SIZE, shuffle=True,
                      collate_fn=generate_batch)
    for i, (text, offsets, cls) in enumerate(data):
        optimizer.zero_grad()
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        output = model(text, offsets)
        loss = criterion(output, cls)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (output.argmax(1) == cls).sum().item()

    # Adjust the learning rate
    scheduler.step()

    return train_loss / len(sub_train_), train_acc / len(sub_train_)

def test(data_):
    loss = 0
    acc = 0
    data = DataLoader(data_, batch_size=BATCH_SIZE, collate_fn=generate_batch)
    for text, offsets, cls in data:
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        with torch.no_grad():
            output = model(text, offsets)
            loss = criterion(output, cls)
            loss += loss.item()
            acc += (output.argmax(1) == cls).sum().item()

    return loss / len(data_), acc / len(data_)

In [56]:
import time
from torch.utils.data.dataset import random_split
N_EPOCHS = 5
min_valid_loss = float('inf')

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=4.0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

train_len = int(len(train_data) * 0.95)
sub_train_, sub_valid_ = \
    random_split(train_data, [train_len, len(train_data) - train_len])


for epoch in range(N_EPOCHS):

    start_time = time.time()
    train_loss, train_acc = train_func(sub_train_)
    valid_loss, valid_acc = test(sub_valid_)

    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60

    print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'torchtext.data.example.Example'>